In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import json
 

#PyMySQL
import pymysql
pymysql.install_as_MySQLdb()
from config import myPass

In [2]:
csv_file = "Resources/listing-of-active-businesses3.csv"
active_businesses_df = pd.read_csv(csv_file, encoding="UTF-8")
active_businesses_df.head()

,BUSINESS NAME,STREET ADDRESS,CITY,ZIP CODE
0,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350
1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,2015 W TEMPLE STREET,LOS ANGELES,90026-4913
2,SPRINGBOARD NON-PROFIT CONSUMER CREDIT MANAGEMENT,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828
3,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,3806 W PICO BLVD,LOS ANGELES,90019-4304
4,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,3115 VENICE BLVD,LOS ANGELES,90019-6238


In [3]:
list(active_businesses_df)

['BUSINESS NAME', 'STREET ADDRESS', 'CITY', 'ZIP CODE']

In [4]:
active_businesses2_df = active_businesses_df[["BUSINESS NAME","STREET ADDRESS","CITY","ZIP CODE"]].copy()
active_businesses2_df.head()

renamed_business_df = active_businesses2_df.rename(columns = {"BUSINESS NAME":"business_name","STREET ADDRESS":"street", "CITY":"city","ZIP CODE":"zip_code"})
renamed_business_df.head()

,business_name,street,city,zip_code
0,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350
1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,2015 W TEMPLE STREET,LOS ANGELES,90026-4913
2,SPRINGBOARD NON-PROFIT CONSUMER CREDIT MANAGEMENT,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828
3,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,3806 W PICO BLVD,LOS ANGELES,90019-4304
4,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,3115 VENICE BLVD,LOS ANGELES,90019-6238


In [5]:
#bike_data_json_file = "Resources/socrata_metadata.json"
#bike_data_df = pd.read_json(bike_data_json_file)
#bike_data_df.head()

with open('Resources/bikes.json') as json_data:
   data = json.load(json_data)


print(json.dumps(data, indent=4, sort_keys=True))

#print(data.json())
#df = pd.DataFrame.from_dict(data, orient='index')
#df.he

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -118.25854,
                    34.0485
                ],
                "type": "Point"
            },
            "properties": {
                "addressCity": "Los Angeles",
                "addressState": "CA",
                "addressStreet": "723 Flower Street",
                "addressZipCode": "90017",
                "bikesAvailable": 21,
                "classicBikesAvailable": 21,
                "closeTime": "23:58:00",
                "docksAvailable": 4,
                "electricBikesAvailable": 0,
                "eventEnd": null,
                "eventStart": null,
                "hasGeofence": false,
                "isEventBased": false,
                "isVirtual": false,
                "isVisible": false,
                "kioskConnectionStatus": "Active",
                "kioskId": 3005,
                "kioskPublicStatus": "Active",
                "kio

In [6]:
Address = []
Names = []
City = []
State = []
Zip_Code = []

features = data["features"]

for feature in features:
    Address.append(feature["properties"]["addressStreet"])
    Names.append(feature["properties"]["name"])
    City.append(feature["properties"]["addressCity"])
    State.append(feature["properties"]["addressState"])
    Zip_Code.append(feature["properties"]["addressZipCode"])
    
#print(Address)
#print(len(Address))
#print(Names)
#print(len(Names))
#print(City)
#print(len(City))
#print(State)
#print(len(State))
#print(Zip_Code)
#print(len(Zip_Code))

In [7]:
bike_data_df = pd.DataFrame({"street":Address, "name":Names, "city":City, "state":State, "zip_code":Zip_Code})
bike_data_df.head()

,street,name,city,state,zip_code
0,723 Flower Street,7th & Flower,Los Angeles,CA,90017
1,729 S Olive Street,Olive & 8th,Los Angeles,CA,90014
2,557 S 5th Street,5th & Grand,Los Angeles,CA,90071
3,865 S Figueroa Street,Figueroa & 9th,Los Angeles,CA,90017
4,401 East 11th Street,11th & Maple,Los Angeles,CA,90015


In [8]:
rds_connection_string = "root:mick1960@127.0.0.1/LASTUFF_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [9]:
renamed_business_df.to_sql(name='buisness', con=engine, if_exists='replace',index=False)

In [10]:
bike_data_df.to_sql(name="bikes", con=engine, if_exists="replace", index=False)